**1. MC-TCI**

In [9]:
import TensorCrossInterpolation as TCI

function f(v)
    j = v[1]
    k = v[2]
    l = v[3]
    
    NSq = N^2
    
    term1 = cos(2*π*(j*k/NSq)) * sin(2*π*(j*l/NSq)) * exp(-j/N)
    term2 = sin(2*π*(j*k/NSq)) * cos(2*π*(j*l/NSq)) * exp(-j^2/NSq)

    return term1 + term2
end


function exact_sum()
    sum = 0.0
    
    for j in 1:N
        for k in 1:N
            for l in 1:N
                sum += f([j, k, l])
            end
        end
    end
    return sum / N^3
end


function mc_tci_sum(M)

    tciSumLst = zeros(M)
    localdims = fill(N, 2)

    for i in 1:M
        j = rand(1:N)
        g(v) = f([j, v[1], v[2]]) #the method of fix one variable
        
        tci_2d, _, _ = TCI.crossinterpolate2(Float64, g, localdims; tolerance = 1e-8 )
        
        mps = tci_2d.sitetensors
        
        sumK = dropdims(sum(mps[1],dims=2),dims=2) #(r1 X N X r2) -> (r1 X 1 X r2) -> (r1 x r2)
        sumL = dropdims(sum(mps[2],dims=2),dims=2)

        resultMatrix = sumK * sumL #1x1 matrix
        
        tciSumLst[i] = resultMatrix[1] / N^2
    end
    
    return sum(tciSumLst) / length(tciSumLst)
end

N = 20

# conduct exact sum
exactSum = exact_sum()
println("exact sum = $exactSum")

MLst = [10_000]

@time for M in MLst
    X = 10
    acc = 0
    accSq = 0

    for _ in 1:X
        mctciSum = mc_tci_sum(M)

        acc += mctciSum
        accSq += mctciSum^2
    end

    var = (accSq / X - (acc / X)^2)
    stdErr = sqrt(var / X)

    println("\nnumber of j : $M")
    println("MC-TCI stdErr : $stdErr" )
    println("MC-TCI result : ", acc/X)
end

exact sum = 0.18689630762777193

number of j : 10000
MC-TCI stdErr : 0.0008375184604249656
MC-TCI result : 0.18584944439979564
366.287028 seconds (7.87 G allocations: 156.418 GiB, 7.92% gc time, 0.34% compilation time)


In [14]:
import TensorCrossInterpolation as TCI
using Random

function f(v)
    j = v[1]
    k = v[2]
    l = v[3]
    
    NSq = N^2
    
    term1 = cos(2*π*(j*k/NSq)) * sin(2*π*(j*l/NSq)) * exp(-j/N)
    term2 = sin(2*π*(j*k/NSq)) * cos(2*π*(j*l/NSq)) * exp(-j^2/NSq)

    return term1 + term2
end


function exact_sum()
    sum = 0.0
    
    for j in 1:N
        for k in 1:N
            for l in 1:N
                sum += f([j, k, l])
            end
        end
    end
    return sum / N^3
end


function mc_tci_sum(K,L,M)
    accMc = 0.0

    tciSumLst = zeros(K)
    localdims = fill(N, 2)

    for i in 1:M
        arr = collect(1:N)
        sArr = shuffle(arr)
        for k in 1:K
            j = sArr[k]
            g(v) = f([j, v[1], v[2]]) #the method of fix one variable
            
            tci_2d, _, _ = TCI.crossinterpolate2(Float64, g, localdims; tolerance = 1e-8 )
            
            mps = tci_2d.sitetensors
            
            sumK = dropdims(sum(mps[1],dims=2),dims=2) #(r1 X N X r2) -> (r1 X 1 X r2) -> (r1 x r2)
            sumL = dropdims(sum(mps[2],dims=2),dims=2)

            resultMatrix = sumK * sumL #1x1 matrix
            
            tciSumLst[k] = resultMatrix[1] / N^2
        end

        for l in 1:L
            j = rand(1:K)
            accMc += tciSumLst[j]
        end
    end
    
    return  accMc / (M*L)
end

N = 20

# conduct exact sum
exactSum = exact_sum()
println("exact sum = $exactSum")

MLst = [100]

K = 20
L = 1000
M = 10

@time for M in MLst
    X = 10
    acc = 0
    accSq = 0

    for i in 1:X
        mctciSum = mc_tci_sum(K,L,M)
        #println("$i mctciSum = $mctciSum")
        acc += mctciSum
        accSq += mctciSum^2
    end

    var = (accSq / X - (acc / X)^2)
    stdErr = sqrt(var / X)

    println("\nnumber of j : $M")
    println("MC-TCI stdErr : $stdErr" )
    println("MC-TCI result : ", acc/X)
end

exact sum = 0.18689630762777193

number of j : 100
MC-TCI stdErr : 0.00018881979711215548
MC-TCI result : 0.18712841390319748
 75.453559 seconds (1.58 G allocations: 31.356 GiB, 7.99% gc time, 1.64% compilation time)
